In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from torch import nn
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
import os   # for os.path.join
import numpy as np
import matplotlib.pyplot as plt

In [ ]:


class AUTOMAPModel(pl.LightningModule):
    def __init__(self, n_H0, n_W0):
        super(AUTOMAPModel, self).__init__()
        self.n_H0 = n_H0
        self.n_W0 = n_W0

        # Define the model architecture
        self.fc1 = nn.Linear(n_H0 * n_W0 * 2, n_H0 * n_W0)
        self.fc2 = nn.Linear(n_H0 * n_W0, n_H0 * n_W0)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, padding='same')
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding='same')
        self.conv3 = nn.Conv2d(64, 1, kernel_size=7, padding='same')

        # Activation and Regularization
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self, x):
        # Reshape x to match the input dimensions
        x = x.view(-1, self.n_H0 * self.n_W0 * 2)

        # Forward pass
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        # Reshape for convolutional layers
        x = x.view(-1, 1, self.n_H0, self.n_W0)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x.squeeze(1)  # Remove channel dimension for the output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.mean((y_hat - y) ** 2)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=0.001)
        return optimizer


def load_data(X_train, Y_train, batch_size=64):
    # Convert numpy arrays to PyTorch tensors
    tensor_x = torch.Tensor(X_train)  # transform to torch tensor
    tensor_y = torch.Tensor(Y_train)

    # Create a dataset and dataloader
    my_dataset = TensorDataset(tensor_x, tensor_y)  # create your datset
    # create your dataloader
    my_dataloader = DataLoader(my_dataset, batch_size=batch_size)
    return my_dataloader








In [16]:
import os
import cv2
from PIL import Image
# Define the directory where your images are located
image_dir = '/home/avi/data/imagenet_20/train1-2'
# Define the output directories where processed images will be saved
automap_data_dir = '/home/avi/data/imagenet_20/automap_data'

# Create the output directories if they don't exist
os.makedirs(automap_data_dir, exist_ok=True)

# Initialize counters to keep track of the number of files processed
files_count = 0
required_files_count = 1300
image_size_h = 110
image_size_w = 110



# Iterate over each subdirectory in the image directory
for root, dirs, files in os.walk(image_dir):
    # Iterate over each file in the current subdirectory
    for filename in files:
        # Check if the maximum file count has been reached for train_dir
        if files_count >= required_files_count:
            break

        # Open the image file
        with Image.open(os.path.join(root, filename)) as img:
            # Crop to the central 256x256 pixels
            width, height = img.size
            left = (width - 256) / 2
            top = (height - 256) / 2
            right = (width + 256) / 2
            bottom = (height + 256) / 2
            img_cropped = img.crop((left, top, right, bottom))

            # Subsample to 128x128
            img_resized = img_cropped.resize((image_size_h, image_size_w))

            # Convert to YCbCr and keep only the Y (luminance) channel
            img_y, _, _ = img_resized.convert('YCbCr').split()

            # Determine whether to save the image in train_dir or test_dir
            if files_count < required_files_count:
                img_y.save(os.path.join(automap_data_dir, filename))
                files_count += 1

        # Check if the maximum file count has been reached for train_dir
        if files_count >= required_files_count:
            break

In [17]:
import random
import shutil
import os

# Define the directories
automap_data_dir = '/home/avi/data/imagenet_20/automap_data'
automap_train_dir = '/home/avi/data/imagenet_20/automap_train'
automap_test_dir = '/home/avi/data/imagenet_20/automap_test'

# Create the output directories if they don't exist
os.makedirs(automap_train_dir, exist_ok=True)
os.makedirs(automap_test_dir, exist_ok=True)

# Get the list of files in the automap_train_dir
files = os.listdir(automap_train_dir)
# Delete each file in the automap_train_dir
for file in files:
    file_path = os.path.join(automap_train_dir, file)
    os.remove(file_path)

# Get the list of files in the automap_train_dir
files = os.listdir(automap_test_dir)
# Delete each file in the automap_train_dir
for file in files:
    file_path = os.path.join(automap_test_dir, file)
    os.remove(file_path)



# Get the list of files in the automap_data_dir
files = os.listdir(automap_data_dir)

# Randomly select 10,000 files
selected_files = random.sample(files, 1000)

# Move the selected files to automap_train_dir
for file in selected_files:
    src = os.path.join(automap_data_dir, file)
    dst = os.path.join(automap_train_dir, file)
    shutil.move(src, dst)

# Move the remaining files to automap_test_dir
for file in files:
    if file not in selected_files:
        src = os.path.join(automap_data_dir, file)
        dst = os.path.join(automap_test_dir, file)
        shutil.move(src, dst)

In [15]:
import os
import torch
from torchvision.transforms import functional as F
from PIL import Image

# Define the directory where the images are located
automap_train_dir = '/home/avi/data/imagenet_20/automap_train'

# Define the number of rotations
num_rotations = 4

# Get the list of files in the automap_train_dir
files = os.listdir(automap_train_dir)

# Create an empty tensor to store the augmented images
augmented_images = torch.empty((len(files) * num_rotations, 128, 128), dtype=torch.float32)

# Iterate over each file in the automap_train_dir
for i, file in enumerate(files):
    # Open the image file
    with Image.open(os.path.join(automap_train_dir, file)) as img:
        # Convert the image to a PyTorch tensor
        img_tensor = F.to_tensor(img)

        # Rotate the image and store the augmented images
        for j in range(num_rotations):
            rotated_img_tensor = F.rotate(img_tensor, 90 * (j + 1))
            augmented_images[i * num_rotations + j] = rotated_img_tensor

# Print the shape of the augmented images tensor
print(augmented_images.shape)


RuntimeError: The expanded size of the tensor (128) must match the existing size (110) at non-singleton dimension 1.  Target sizes: [128, 128].  Tensor sizes: [110, 110]

In [ ]:
import matplotlib.pyplot as plt

for i in range(4, 8):
    plt.subplot(2, 2, i - 3)
    plt.imshow(augmented_images[i], cmap='gray')
    plt.axis('off')

plt.show()


In [6]:
fft_images = torch.fft.fft2(augmented_images)

real_part = fft_images.real
imaginary_part = fft_images.imag
sensor_data = torch.stack([real_part, imaginary_part], dim=-1)

# Print the shape of the stacked images tensor
print(sensor_data.shape)



torch.Size([40000, 128, 128, 2])


In [13]:
# Example usage
n_H0, n_W0 = 128, 128  # Example dimensions, modify as per your dataset
model = AUTOMAPModel(n_H0, n_W0)

# Assuming X_train and Y_train are available as numpy arrays
X_train = sensor_data
Y_train = augmented_images
train_dataloader = load_data(X_train, Y_train, batch_size=1)


: 

In [12]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | fc1   | Linear | 536 M 
1 | fc2   | Linear | 268 M 
2 | conv1 | Conv2d | 1.7 K 
3 | conv2 | Conv2d | 102 K 
4 | conv3 | Conv2d | 3.1 K 
5 | tanh  | Tanh   | 0     
6 | relu  | ReLU   | 0     
---------------------------------
805 M     Trainable params
0         Non-trainable params
805 M     Total params
3,221.786 Total estimated model params size (MB)
/home/avi/virtual_env/automap/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/40000 [00:00<?, ?it/s] 

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacity of 11.73 GiB of which 1.43 GiB is free. Including non-PyTorch memory, this process has 10.28 GiB memory in use. Of the allocated memory 10.02 GiB is allocated by PyTorch, and 5.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)